# Momentum Indicators - Apple Stock 2023-2025

Testing momentum indicators: RSI, MACD, ADX, ROC, MOM, Stochastic.

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path
sys.path.insert(0, str(Path('../..').resolve()))

from indicators.momentum import (
    calculate_rsi,
    calculate_macd,
    calculate_adx,
    calculate_roc,
    calculate_mom,
    calculate_stochastic
)

In [ ]:
# Fetch Apple data
df = yf.download('AAPL', start='2023-01-01', end='2025-10-01', auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

## 1. RSI - Relative Strength Index

In [ ]:
df['rsi'] = calculate_rsi(df, period=14)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'RSI (14)'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['rsi'], name='RSI', line=dict(color='blue')), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, annotation_text="Overbought")
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Oversold")
fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="RSI - Momentum Oscillator")
fig.show()

print(f"RSI mean: {df['rsi'].mean():.2f}")
print(f"Oversold periods (<30): {(df['rsi'] < 30).sum()} days")
print(f"Overbought periods (>70): {(df['rsi'] > 70).sum()} days")

## 2. MACD - Moving Average Convergence Divergence

In [ ]:
macd, signal, histogram = calculate_macd(df, fastperiod=12, slowperiod=26, signalperiod=9)
df['macd'] = macd
df['macd_signal'] = signal
df['macd_hist'] = histogram

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'MACD Line & Signal', 'MACD Histogram'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['macd'], name='MACD', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['macd_signal'], name='Signal', 
                        line=dict(color='red', dash='dash')), row=2, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)

fig.add_trace(go.Bar(x=df['date'], y=df['macd_hist'], name='Histogram',
                    marker_color=['green' if val >= 0 else 'red' for val in df['macd_hist']]), row=3, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=3, col=1)

fig.update_layout(height=800, title_text="MACD - Trend Following Momentum")
fig.show()

print("MACD normalized by ATR for better cross-stock comparison.")

## 3. ADX - Average Directional Index

In [ ]:
df['adx'] = calculate_adx(df, period=14)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'ADX (14) - Trend Strength'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['adx'], name='ADX', line=dict(color='purple')), row=2, col=1)
fig.add_hline(y=25, line_dash="dash", line_color="orange", row=2, col=1, annotation_text="Strong Trend")
fig.add_hline(y=20, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="ADX - Trend Strength Indicator")
fig.show()

print(f"ADX mean: {df['adx'].mean():.2f}")
print(f"Strong trend periods (>25): {(df['adx'] > 25).sum()} days")

## 4. ROC - Rate of Change

In [ ]:
df['roc'] = calculate_roc(df, period=10)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'ROC (10) - Rate of Change %'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['roc'], name='ROC', 
                        fill='tozeroy', line=dict(color='teal')), row=2, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="ROC - Price Momentum")
fig.show()

print(f"ROC mean: {df['roc'].mean():.2f}%")
print(f"ROC std: {df['roc'].std():.2f}%")

## 5. MOM - Momentum

In [ ]:
df['mom'] = calculate_mom(df, period=10)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'MOM (10) - Absolute Momentum'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['mom'], name='MOM',
                        fill='tozeroy', line=dict(color='brown')), row=2, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="MOM - Raw Price Momentum")
fig.show()

print(f"MOM mean: ${df['mom'].mean():.2f}")
print(f"MOM std: ${df['mom'].std():.2f}")

## 6. Stochastic Oscillator

In [ ]:
stoch_k, stoch_d = calculate_stochastic(df, fastk_period=14, slowk_period=3, slowd_period=3)
df['stoch_k'] = stoch_k
df['stoch_d'] = stoch_d

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'Stochastic Oscillator (%K, %D)'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['stoch_k'], name='%K', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['stoch_d'], name='%D', 
                        line=dict(color='red', dash='dash')), row=2, col=1)
fig.add_hline(y=80, line_dash="dash", line_color="red", row=2, col=1, annotation_text="Overbought")
fig.add_hline(y=20, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Oversold")
fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="Stochastic - Range-Based Momentum")
fig.show()

print(f"Stochastic %K mean: {df['stoch_k'].mean():.2f}")
print(f"Oversold periods (<20): {(df['stoch_k'] < 20).sum()} days")
print(f"Overbought periods (>80): {(df['stoch_k'] > 80).sum()} days")

## Combined View - All Momentum Indicators

In [ ]:
fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.03,
                    subplot_titles=(
                        'Price',
                        'RSI',
                        'MACD',
                        'ADX',
                        'ROC vs MOM',
                        'Stochastic'
                    ))

# Row 1: Price
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

# Row 2: RSI
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi'], name='RSI', line=dict(color='blue')), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

# Row 3: MACD
fig.add_trace(go.Scatter(x=df['date'], y=df['macd'], name='MACD', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['macd_signal'], name='Signal',
                        line=dict(color='red', dash='dash')), row=3, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=3, col=1)

# Row 4: ADX
fig.add_trace(go.Scatter(x=df['date'], y=df['adx'], name='ADX', line=dict(color='purple')), row=4, col=1)
fig.add_hline(y=25, line_dash="dash", line_color="orange", row=4, col=1)

# Row 5: ROC vs MOM (normalized)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
roc_norm = scaler.fit_transform(df[['roc']].fillna(0))
mom_norm = scaler.fit_transform(df[['mom']].fillna(0))

fig.add_trace(go.Scatter(x=df['date'], y=roc_norm.flatten(), name='ROC (norm)',
                        line=dict(color='teal')), row=5, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=mom_norm.flatten(), name='MOM (norm)',
                        line=dict(color='brown', dash='dash')), row=5, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=5, col=1)

# Row 6: Stochastic
fig.add_trace(go.Scatter(x=df['date'], y=df['stoch_k'], name='%K', line=dict(color='blue')), row=6, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['stoch_d'], name='%D',
                        line=dict(color='red', dash='dash')), row=6, col=1)
fig.add_hline(y=80, line_dash="dash", line_color="red", row=6, col=1)
fig.add_hline(y=20, line_dash="dash", line_color="green", row=6, col=1)

fig.update_layout(height=1400, title_text="Momentum Indicators - Complete Overview")
fig.show()

print("\n=== Momentum Summary ===")
print(f"RSI: mean={df['rsi'].mean():.2f}, oversold={((df['rsi'] < 30).sum())} days")
print(f"ADX: mean={df['adx'].mean():.2f}, strong trend={(df['adx'] > 25).sum()} days")
print(f"Stochastic: mean={df['stoch_k'].mean():.2f}, oversold={(df['stoch_k'] < 20).sum()} days")